# The H-R diagram
This notebook will explore the HR diagram, perhaps the most important figure in astronomy, and a classic example of the power of data visualization.

When we look at the Sky, some stars are bright and some faint. This is due not only to their intrinsic properties. but also to their distance from us. If we want to study the intrinsic properties of stars we need to have first determined their distances.

In [ ]:
from astropy.io import ascii
from astropy.table import Table
from astropy import units as u

We'll be using a catalog of stars known as the [HGY database](http://www.astronexus.com/hyg). It is the combination of three surveys which have meadured distances. In the file I've provided in this repository I've filtered that catalog to only include columns we need and to also remove entries without measured distances and colors.

In [ ]:
starTable = ascii.read('HGY_dist.dat')
starTable

In [ ]:
# plotting imports
import matplotlib.pyplot as plt
%matplotlib inline
#import seaborn
import numpy as np
from astropy.coordinates import Angle

In [ ]:
import seaborn


## Plotting the Skymap
Let's see where those stars are on the sky

In [ ]:
#seaborn.set_style(("darkgrid"))
fig = plt.figure (figsize=(13,6))
ax = fig.add_subplot(111,projection="mollweide")
plt.scatter(Angle(starTable[1:]['ra'],u.hr).wrap_at(180.*u.deg).radian,Angle(starTable[1:]['dec'],u.deg).radian,s=1,edgecolors='none')

OK, well se can see our Galaxy in that plot, but not much else. Let's do another map, this time only showing stars brighter than 4th magnitude, and also scaling the stars bytheir brightness.

In [ ]:
fig = plt.figure (figsize=(13,6))
ax = fig.add_subplot(111,projection="mollweide")
plt.scatter(Angle(starTable[1:]['ra'],u.hr).wrap_at(180.*u.deg).radian,Angle(starTable[1:]['dec'],u.deg).radian,s=(2.0*(4.0-starTable[1:]['mag'])))

### Variable distributions
To start let's see how the quantities we'll be working with are distributed.

In [ ]:
fig = plt.figure (figsize=(12,9))
ax = fig.add_subplot(311)
ax.hist(starTable['dist'],bins=100)
ax.set_xlabel("Distance(pc)")
a2 = fig.add_subplot(312)
a2.hist(starTable['absmag'],bins=100)
a2.set_xlabel("M")
a3 = fig.add_subplot(313)
a3.hist(starTable['ci'],bins=100)
a3.set_xlabel("B-V")

## The HR Diagram
Simply plotting Color vs Absolute Magniture gives us the HR Diagram

In [ ]:
#seaborn.set_style(("darkgrid"))
fig = plt.figure (figsize=(12.5, 7.5))
plt.gca().invert_yaxis()
plt.xlim(-0.5,2.5)
plt.ylim(17,-7)
plt.title('H-R Diagram',fontsize=26)
plt.ylabel('Absolute Magnitude (V)')
plt.xlabel('Color (B-V)')
plt.scatter(starTable['ci'] ,starTable['absmag'],s=.5,edgecolors='none')

In [ ]:
#seaborn.set_style(("darkgrid"), {"axes.facecolor": ".2"})
fig = plt.figure (figsize=(12.5, 7.5))
plt.gca().invert_yaxis()
plt.xlim(-0.5,2.5)
plt.ylim(17,-7)
plt.title('H-R Diagram',fontsize=26)
plt.ylabel('Absolute Magnitude (V)')
plt.xlabel('Color (B-V)')
plt.scatter(starTable['ci'] ,starTable['absmag'],s=.5,cmap='coolwarm',c=starTable['ci'],edgecolors='none',vmin=0.0,vmax=1.5)

## HR Diagram and Stellar Evolution
The grouping in the HR diagram led to the recogntion that the different regions in the diagram cooresponeded to an evolutionary sequence. We now have detailed models for how a star of a given mass evolves over its lifetime. [Here](http://www.epantaleo.com/wp-content/uploads/2015/10/HR_diagram_d3.html) is a visualization of the evolution of a 100 star cluster over 5 billions years. This was produced by Ester Panatelo, Aaron Geller and myself based on simulations run by Aaron Geller using the NBODY6 code which includes both gravitational interactions and steller evolution.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("qJMom80Qdc8")

## Color Magnitude Diagram of a Globular Clusters
The color magnitude diagram is very similar to the HR diagram, except it is plotted using apparent rather than absolute magnitudes. If we were to construct this for a random sample of stars it would be a mess, however a cluster of stars are approximately at the same distance, so there is just a constant shift from the HR diagram based on the distance to the cluster.

##### We can conect to Worldwide Telescope to see the imagery of the cluster we are working with, Palomar 5

In [ ]:
from pywwt.windows import WWTWindowsClient
my_wwt = WWTWindowsClient()

In [ ]:
# Tell WWT to Fly to the position of the cluster
my_wwt.change_mode('Sky',fly_to=[-0.1082,229.0128/15,1,0.,0.])

The catalog pal5.csv was created by soing a radial search for objects in the Sloan Digital Sky Survey [SDSS](http://sdss.org) for all abjects within three arcminuted of the center of the cluster.

In [ ]:
pal5 = ascii.read('pal5.csv')
pal5

Oops that contains everything SDSS identified in the direction of the cluster, we only want stars (not galaxies). We just keep things with type=6 (6 means star in this catalog, 3 galaxy).

In [ ]:
mask = pal5['type']==6
pal5_stars = pal5[mask]
pal5_stars

##### OK, now lets's see the catalog in WWT

In [ ]:
#Set up WWT layer
new_layer = my_wwt.new_layer("Sky", "Palomar 5", pal5_stars.colnames)
#Set visualization parameters in WWT
props_dict = {"CoordinatesType":"Spherical",\
              "MarkerScale":"Screen",\
              "PlotType":"Circle",\
              "PointScaleType":"Constant",\
              "ScaleFactor":"16",\
              "RaUnits":"Degrees",\
              "TimeSeries":"False"}
new_layer.set_properties(props_dict)
#Send data to WWT client
new_layer.update(data=pal5_stars, purge_all=True, no_purge=False, show=True)


##### And now let's plot the color-magnitude diagram

In [ ]:
#seaborn.set_style(("darkgrid"))
fig = plt.figure (figsize=(7.5, 5))
plt.gca().invert_yaxis()
plt.xlim(-0.5,2.0)
plt.ylim(25,16)
plt.title('Color-Magnitude Diagram for Pal5',fontsize=26)
plt.ylabel('Apparent Magnitude (g)')
plt.xlabel('color (g-r)')
plt.scatter(pal5_stars['g']-pal5_stars['r'] ,pal5_stars['g'],s=2,cmap='coolwarm',edgecolors='none',vmin=0.0,vmax=1.0)

## Things for you to try:
Now we can figure out a lot of things with HR color-magnitude diagrams. I'll get you started with the following exercises, but you'll have to do most of the work yourself.

#### Finding the Distance to Pal 5
The color Magnitude Diagram uses apparent magnitudes instead of  absolute magnitudes. The difference between the apparent and absolute magnitude is known as the distance modulus = (m - M). Extimate the distance modulus from the shift in the two diagrams to esitmate the distance to Pal 5. Now things are a little complicared by the fact that we are using different magnitude systems (B and V vs. SDSS g andr). However the shifts between the two systems and can be ignored for this rough estimate.

In [ ]:
# This is the code for converting the distance Modulus to a distance 
from astropy.coordinates import Distance
Distance(distmod=10.0)

#### Which is older pal 5 or pal 3
Construct the color-magnitude of Pal 3 (RA = 151.3801 deg, dec= 0.072 deg). By comparing the location of their main sequence turnoffs, can you figure out which cluster is older.

I've already downloaded the data for you, you can download it from the repository like this:

In [ ]:
# Load Pal 3 data
pal3objects = ascii.read('pal3.csv')
pal3objects

#### Make an HR Diagram from Gaia Data Release 1 (2 million stars!)
The [Gaia](http://sci.esa.int/gaia/) mission had recently released a sample of 2 million star distances, a number that dwarfs what we have been using. But that is nothing in April 2018 they will release a sample containing over one billion!!!

Now this will be the ultimate HR diagram. I'll provide you with the code to download the catalog to get you started. Good  luck!

In [ ]:
# We will download the catalog from the VizieR catalog service
from astroquery.vizier import Vizier
v = Vizier()
v.ROW_LIMIT = -1 # Without this there is a limit
catalogs = v.get_catalogs('I/337/tgas')
GaiaStars=catlogs[0]